##  customer_data
##  customer_id  name tables
##  orders_id :  same as transactions except customer_id

### when added item in transaction store info will be updated and customer_id_order table will be updated

In [28]:
import sqlite3
import pandas as pd
import datetime

In [29]:
con = sqlite3.connect('../new_data.db')
cursorObj = con.cursor()

In [30]:
def find_next_invoice_no(transactions_1):
    
    invoice_1 = transactions_1['Invoice']#.sort_values()
    invoice_1 = sorted(invoice_1)
    if(len(invoice_1) == 0):
        return str(1)
    count = 0
    for i in invoice_1:
        if (i[0] == '1' or i[0] == '2' or i[0] == '3' or i[0] == '4' or i[0] == '5' or i[0] == '6' or i[0] == '7' or i[0] == '8' or i[0] == '9' or i[0] == '0' ):
            count = count + 1
        else:
            break

    if(count>0):
        count = count - 1
    # print("last index : ",count,"| value :", invoice_1[count])
    val = int(invoice_1[count])
    # print("next invoice number: ", val+1)
    return str(val+1)


def update_store(retailer_id, stockcode, quantity, cursorObj, con):

    sql = "update store_"+str(retailer_id)+" set Quantity = Quantity - "+str(quantity)+" WHERE StockCode = ?"
    print(retailer_id, stockcode, quantity)
    cursorObj.execute(sql, (stockcode,))
    con.commit()
    print('updated store data of retailer')

    

def insert_into_transactions(stockcode, description, quantity, price, country, customer_id, retailer_id, con, cursorObj):
   
    #name of retailor's transcations table
    val = cursorObj.execute('select Transactions_table from retailor_data where retailor_id = ?', (retailer_id,))

    transactions_table_name = val.fetchall()
    transactions_table_name = transactions_table_name[0][0]
    print("entered values in transactions_" +  str(retailer_id))

    #updating store_info of retailer
    
    sql = "update store_"+str(retailer_id)+" set Quantity = Quantity - "+str(quantity)+" WHERE StockCode = ?"
    cursorObj.execute(sql, (stockcode,))
    con.commit()
    print('updated store data of retailer_'+str(retailer_id))


    sql = "insert into "+str(transactions_table_name)+" values(?,?,?,?,?,?,?,?);"


    query = "select * from " + transactions_table_name
    #getting complete transaction table
    transactions_table = pd.read_sql_query(query,con)
    
    #Getting invoice number
    invoice = find_next_invoice_no(transactions_table)
    timestamp =  datetime.datetime.now().replace(microsecond = 0)
    
    
    details = (invoice, stockcode, description, quantity , timestamp, price, customer_id, country)
    cursorObj.execute(sql, details)


    #Entering in customer order
    sql = "insert into orders_"+str(customer_id)+" values(?,?,?,?,?,?,?);"
    details = (invoice, stockcode, description, quantity , timestamp, price, country)
    cursorObj.execute(sql, details)
    print("entered the placed order in order table of customer_"+str(customer_id))
    con.commit()

    print('successfully completed transcation!')

In [31]:
def generate_sales(con, cursorObj, df, retailer_id):

    df['Price'] = df['Quantity']*df['Price']
    df['InvoiceDate_formatted'] = pd.to_datetime(df['InvoiceDate'])
    df['just_date'] = df['InvoiceDate_formatted'].dt.date
    dates = df['just_date'].unique()
    price_list = []
    dates_list = []
    for i in dates:
        amount = df[df['just_date'] == i]['Price'].sum()
        dates_list.append(i)
        price_list.append(amount)
    df[df['just_date'] == dates[0]]['Price'].sum()
    df = pd.DataFrame(list(zip(dates_list, price_list)), 
                columns =['Date', 'Price']) 
    df.to_sql('sales_'+str(retailer_id),con, if_exists='replace',index = False , schema = 'schema_name') 
    con.commit()
    
def upload_csv(con, cursorObj, retailer_id, store_info = pd.DataFrame, transactions_info = pd.DataFrame):
    if store_info.empty == False:
        store_info.to_sql("store_"+str(retailer_id), con, index = False, if_exists = 'replace')
        con.commit()
        print("store csv values uploaded in store_"+str(retailer_id))
    else:
        print("store table is empty")
    
    if transactions_info.empty == False:
        transactions_info.to_sql("transactions_"+str(retailer_id), con, index = False, if_exists = 'replace', schema = 'schema_name')
        generate_sales(con, cursorObj, transactions_info, retailer_id)
        print('sales_'+str(retailer_id)+' table also generated based on transactions')
        con.commit()
        print("transactions csv values uploaded in transactions_"+str(retailer_id))
    else:
        print("transaction table is empty")


In [32]:
df2 =  pd.read_sql_query('select * from transactions_1', con)
df  =  pd.read_sql_query('select * from store_1', con)
upload_csv(con, cursorObj, 1, df, df2)

In [34]:
df2 =  pd.read_sql_query('select * from transactions_2', con)
df  =  pd.read_sql_query('select * from store_2', con)
upload_csv(con, cursorObj, 2, df, df2)

store csv values uploaded in store_2
sales_2 table also generated based on transactions
transactions csv values uploaded in transactions_2


# pass variable accordingly

In [70]:
# retailer id  14
upload_csv(con, cursorObj, 14, store_info = df,transactions_info = df2 )
upload_csv(con, cursorObj, 14, transactions_info = df2 )
upload_csv(con, cursorObj, 14, df )
upload_csv(con, cursorObj, 14, )

store csv values uploaded in store_14
transactions csv values uploaded in transactions_14
store table is empty
transactions csv values uploaded in transactions_14
store csv values uploaded in store_14
transaction table is empty
store table is empty
transaction table is empty


In [32]:
insert_into_transactions('85123A','WHITE HANGING HEART T-LIGHT HOLDER', 12, 2.55, 'United Kingdom', 12, 1, con, cursorObj)

entered values in transactions_1
updated store data of retailer_1
entered the placed order in order table of customer_12
successfully completed transcation!


In [3]:
customer_data = pd.DataFrame(columns=['customer_id', 'Name', 'order_table'])


In [24]:
customer_data = customer_data.append({'customer_id':3,  'Name':'Aakash',  'order_table':'orders_3'
},    ignore_index = True)


In [25]:
customer_data = customer_data.append({'customer_id':4,  'Name':'Ashutosh',  'order_table':'orders_4'
},    ignore_index = True)

In [26]:
# customer_data.drop([0], inplace=True)
customer_data

,customer_id,Name,order_table
0,3,Aakash,orders_3
1,4,Ashutosh,orders_4


In [27]:
customer_data.to_sql('customer_data', con)

In [28]:
con.commit()

# Making their order table

In [31]:
orders_3 = pd.DataFrame(columns=['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price','Country'])
orders_4 = pd.DataFrame(columns=['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price','Country'])


In [32]:
# orders_3 = orders_3.append({'Invoice': , 'StockCode': , 'Description':, 'Quantity': , 'InvoiceDate': , 'Price': ,'Country': },    ignore_index = True)


In [9]:
cursorObj.execute('select * from transactions_1')

OperationalError: database is locked

In [3]:
## customer_id , st
transactions_1 = pd.read_sql_query('select * from transactions_1', con)
transactions_2 = pd.read_sql_query('select * from transactions_2', con)

DatabaseError: Execution failed on sql 'select * from transactions_1': database is locked

In [5]:
transactions_1.columns

Index(['index', 'Invoice', 'StockCode', 'Description', 'Quantity',
       'InvoiceDate', 'Price', 'Customer ID', 'Country'],
      dtype='object')

In [6]:
# invoice_1 = transactions_1['Invoice']#.sort_values()
# invoice_1 = sorted(invoice_1)
# invoice_2 = transactions_2['Invoice']#.sort_values()
# invoice_2 = sorted(invoice_2)

In [7]:
# count = 0
# for i in invoice_1:
#     if (i[0] == '1' or i[0] == '2' or i[0] == '3' or i[0] == '4' or i[0] == '5' or i[0] == '6' or i[0] == '7' or i[0] == '8' or i[0] == '9' or i[0] == '0' ):
#         count = count + 1
#     else:
#         break

# if(count>0):
#     count = count - 1
# print("last index : ",count,"| value :", invoice_1[count])

In [17]:
def find_next_invoice_no(transactions_1):
    
    invoice_1 = transactions_1['Invoice']#.sort_values()
    invoice_1 = sorted(invoice_1)
    count = 0
    for i in invoice_1:
        if (i[0] == '1' or i[0] == '2' or i[0] == '3' or i[0] == '4' or i[0] == '5' or i[0] == '6' or i[0] == '7' or i[0] == '8' or i[0] == '9' or i[0] == '0' ):
            count = count + 1
        else:
            break

    if(count>0):
        count = count - 1
    print("last index : ",count,"| value :", invoice_1[count])
    val = int(invoice_1[count])
    print("next invoice number: ", val+1)
    return str(val+1)
    


In [33]:
val_1 = find_next_invoice_no(transactions_1)
val_2 = find_next_invoice_no(transactions_2)

last index :  397923 | value : 581587
next invoice number:  581588
last index :  805607 | value : 581587
next invoice number:  581588


In [3]:
cursorObj = con.cursor()


In [10]:
import datetime
x = datetime.datetime.now().replace(microsecond = 0)
print(x)

2020-12-08 17:45:01


In [16]:
sql = "insert into transactions_1 values(?,?,?,?,?,?,?,?);"
details = ('1','1','1',1,datetime.datetime.now().replace(microsecond = 0),1,2,'ad')
cursorObj.execute(sql, details)
con.commit()

In [29]:
def find_next_invoice_no(transactions_1):
    
    invoice_1 = transactions_1['Invoice']#.sort_values()
    invoice_1 = sorted(invoice_1)
    if(len(invoice_1) == 0):
        return str(1)
    count = 0
    for i in invoice_1:
        if (i[0] == '1' or i[0] == '2' or i[0] == '3' or i[0] == '4' or i[0] == '5' or i[0] == '6' or i[0] == '7' or i[0] == '8' or i[0] == '9' or i[0] == '0' ):
            count = count + 1
        else:
            break

    if(count>0):
        count = count - 1
    # print("last index : ",count,"| value :", invoice_1[count])
    val = int(invoice_1[count])
    # print("next invoice number: ", val+1)
    return str(val+1)


def update_store(retailer_id, stockcode, quantity, cursorObj, con):

    sql = "update store_"+str(retailer_id)+" set Quantity = Quantity - "+str(quantity)+" WHERE StockCode = ?"
    print(retailer_id, stockcode, quantity)
    cursorObj.execute(sql, (stockcode,))
    con.commit()
    print('updated store data of retailer')

    

def insert_into_transactions(stockcode, description, quantity, price, country, customer_id, retailer_id, con, cursorObj):
   
    #name of retailor's transcations table
    val = cursorObj.execute('select Transactions_table from retailor_data where retailor_id = ?', (retailer_id,))

    transactions_table_name = val.fetchall()
    transactions_table_name = transactions_table_name[0][0]
    print("entered values in transactions_" +  str(retailer_id))

    #updating store_info of retailer
    
    sql = "update store_"+str(retailer_id)+" set Quantity = Quantity - "+str(quantity)+" WHERE StockCode = ?"
    cursorObj.execute(sql, (stockcode,))
    con.commit()
    print('updated store data of retailer_'+str(retailer_id))


    sql = "insert into "+str(transactions_table_name)+" values(?,?,?,?,?,?,?,?);"


    query = "select * from " + transactions_table_name
    #getting complete transaction table
    transactions_table = pd.read_sql_query(query,con)
    
    #Getting invoice number
    invoice = find_next_invoice_no(transactions_table)
    timestamp =  datetime.datetime.now().replace(microsecond = 0)
    
    
    details = (invoice, stockcode, description, quantity , timestamp, price, customer_id, country)
    cursorObj.execute(sql, details)


    #Entering in customer order
    sql = "insert into orders_"+str(customer_id)+" values(?,?,?,?,?,?,?);"
    details = (invoice, stockcode, description, quantity , timestamp, price, country)
    cursorObj.execute(sql, details)
    print("entered the placed order in order table of customer_"+str(customer_id))
    con.commit()

    print('successfully completed transcation!')

In [30]:
insert_into_transactions('85123A','WHITE HANGING HEART T-LIGHT HOLDER', 12, 2.55, 'United Kingdom', 12, 1, con, cursorObj)

entered values in transactions_1
updated store data of retailer_1
entered the placed order in order table of customer_12
successfully completed transcation!


# updating customer and store info

In [ ]:
def update_customer_order():

In [139]:
cursorObj = con.cursor()
q = "select * from store_1 where StockCode = ?"
cursorObj.execute(q, ('85123A',))
print(cursorObj.fetchall())
con.commit()
print('updated store data of retailer')

OperationalError: database is locked

In [137]:
insert_into_transactions?

Signature:
insert_into_transactions(
    stockcode,
    description,
    quantity,
    price,
    country,
    customer_id,
    retailer_id,
)
Docstring: <no docstring>
File:      ~/Desktop/Minor_project/database/create_manage/<ipython-input-134-f5a7832f914a>
Type:      function


In [13]:
insert_into_transactions('85123A','WHITE HANGING HEART T-LIGHT HOLDER', 12, 2.55, 'United Kingdom', 16, 1)

transactions_1
in first func  85123A
1 85123A 12
updated store data of retailer
       Invoice StockCode                         Description  Quantity  \
406841  581597    85123A  WHITE HANGING HEART T-LIGHT HOLDER        12   
406842  581598    85123A  WHITE HANGING HEART T-LIGHT HOLDER        12   
406843  581599    85123A  WHITE HANGING HEART T-LIGHT HOLDER        12   
406844  581600    85123A  WHITE HANGING HEART T-LIGHT HOLDER        12   
406845  581601    85123A  WHITE HANGING HEART T-LIGHT HOLDER        12   

                InvoiceDate  Price  Customer ID         Country  
406841  2020-12-08 14:50:51   2.55          3.0  United Kingdom  
406842  2020-12-08 14:55:29   2.55          3.0  United Kingdom  
406843  2020-12-08 14:56:21   2.55          3.0  United Kingdom  
406844  2020-12-08 14:56:57   2.55          3.0  United Kingdom  
406845  2020-12-08 17:45:05   2.50          3.0  United Kingdom  
last index :  397940 | value : 581601
next invoice number:  581602
successfully

In [140]:
con.close()